********# World University Rankings

Of all the universities in the world, which are the best?

Ranking universities is a difficult, political, and controversial practice. There are hundreds of different national and international university ranking systems, many of which disagree with each other. This dataset contains three global university rankings from very different places.


## Workflow stages

The competition solution workflow goes through seven stages described in the Data Science Solutions book.

1. Question or problem definition.
2. Acquire training and testing data.
3. Wrangle, prepare, cleanse the data.
4. Analyze, identify patterns, and explore the data.
5. Model, predict and solve the problem.
6. Visualize, report, and present the problem solving steps and final solution.
7. Supply or submit the results.


## Question and problem definition

> ได้รู้ว่ามหาวิทยาลัยฮาร์ดวอร์ดเป็นอันดับ1ของทุกปีจากชุดข้อมูลที่ได้ให้ทำการทดสอบโดยดูจากจำนวนนักศึกษาที่เข้าเรียน คะแนนและอันดับการจ้างงานจากศิษย์เก่า



## Workflow goals

**Creating.** สามารถสร้างโมเดลคำถามใหม่จากคำถามเดิมโดยใช้ข้อมูลที่มีอยู่

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

![](http://)## Refactor Release 2017-Jan-29

We are significantly refactoring the notebook based on (a) comments received by readers, (b) issues in porting notebook from Jupyter kernel (2.7) to Kaggle kernel (3.5), and (c) review of few more best practice kernels.

### User comments

- หัวข้อมีความลึกในหลายๆเรื่องของมหาลัยและอีกทั้งยังเข้าใจง่าย (thanks @Wuttidech)
- Correct observation - nearly 30% of the passengers had siblings and/or spouses aboard. (thanks @Reinhard)

### Porting issues

- เนื่องจากมีความแตกต่างทางด้านภาษาทำไห้การอ่านหัวข้อนั้นต้องใช้เวลาในการทำความเข้าใจ


### Best practices

- คุยสรุปหัวข้อหลักที่เราต้องการหากับคนในทีม.
- ใช้การตั้งคำถามที่ดีทำสุดสำหรับตัวเองแล้วนำมารวมกับของทีมแล้วหาคำถามที่ดีที่สุดอีกที.

## Acquire data

รับ Dataset จากเว็ปไซต์ kaggle.com แล้วนำมาอ่านเพื่อหาคำถามโดยเริ่มจากการเอาไปลองสร้างใน Google Datastudio

In [ ]:
cwurData_df = pd.read_csv('../input/cwurData.csv')
timesData_df = pd.read_csv('../input/timesData.csv')

## Analyze by describing data

Pandas also helps describe the datasets answering following questions early in our project.

**คุณสมบัติที่มีอยู่ใน Dataset นี้?**
การแสดงชุดข้อมูลเกี่ยวกับมหาลัยโดยข้อมูลที่มีคือ {world_rank,institution,country,national_rank,quality_of_education} เป็นต้น





In [ ]:
print(cwurData_df.columns.values)

**Which features are categorical?**

- Categorical: world_rank,institution,country,national_rank,quality_of_education.

**Which features are numerical?**

- Continous: total_score,alumni,award,hici

In [ ]:
# preview the data
cwurData_df.head()

**Which features contain blank, null or empty values?**

These will require correcting.

- Cabin > Age > Embarked features contain a number of null values in that order for the training dataset.
- Cabin > Age are incomplete in case of test dataset.

**What are the data types for various features?**

Helping us during converting goal.

- Seven features are integer or floats. Six in case of test dataset.
- Five features are strings (object).

**What is the distribution of numerical feature values across the samples?**

This helps us determine, among other early insights, how representative is the training dataset of the actual problem domain.

- USA เป็น26%ของประเทศทั้งหมด
- คะแนนที่ใช้กำหนดอันดับโลกส่วนใหญ่คะแนนอยู่ในช่วง 43.4-49.02 คะแนน


### Data base analysis


**ความสัมพันธ์.**

เราต้องการทราบว่านอกจากคะแนนที่ปรากฎแล้วเด็กแต่ละคนนั้นมีที่มายังไงโดยข้อมูลที่เราต้องการหาคือ คะแนนขั้นต่ำที่ทางมหาวิทยาลัยต้องการในการสอบเข้า,การจัดกิจกรรมการติวเข้มของมหาวิทยาลัย โดยเราจะนำของแต่ละมหาวิทยาลัยมาเพื่อเปรียบเทียบหาความสัมพันธฺ์

**สิ่งที่ต้องการเมื่อสำเร็จ**

1. 

**ข้อผิดพลาดที่อาจเกิดขึ้น**

1. อันดับอาจจะเปลี่ยนไปถ้าหากนับจากปัจจุบัน
2. การจัดกิจกรรมการติวเข้มอาจไม่มีส่วนช่วยในการหาคำตอบ

**การสร้างใหม่.**

1. เราอาจจะสร้างคะแนนการเข้าขั้นต่ำของแต่ละมหาวิทยาลัย
2. เราอาจจะสร้างหัวข้อ "การจัดกิจกรรมการติวเข้ม" ของแต่ละมหาลัย
3. เราอาจจะสร้างเศรษฐกิจโดยรวมของพื้นที่แต่ละมหาวิทยาลัย

**การจำแนก**

1. ถิ่นฐานที่อยู่
2. คะแนนที่ได้
3. ความต่างกันของคะแนนที่ทางมหาวิทยาลัยคัดเลือกตอนเข้ามหาวิทยาลัย

## Analyze by pivoting features

To confirm some of our observations and assumptions, we can quickly analyze our feature correlations by pivoting features against each other. We can only do so at this stage for features which do not have any empty values. It also makes sense doing so only for features which are categorical (country), ordinal (world_rank).


### Correlating numerical and ordinal features

We can combine multiple features for identifying correlations using a single plot. This can be done with numerical and categorical features which have numeric values.

**Observations.**

- ประเทศส่วนใหญ่ที่ติดอันดับคือUSA
- ประเทศที่เป็นอันดับสองรองจากUSAคือchina
